# Dependências

In [1]:
! pip install psycopg2

In [3]:
! pip install gensim==3.6.0
#!pip install binary gensim==3.6.0

Keyring is skipped due to an exception: 'keyring.backends'


In [4]:
#!pip3 uninstall numpy -y

In [5]:
#!pip install numpy

# Bibliotecas

In [5]:
import pandas as pd
import numpy as np
import psycopg2 as postgres
import psycopg2.extras
import math
from gensim.models import Word2Vec
from tqdm import tqdm
from configparser import ConfigParser
# from sshtunnel import SSHTunnelForwarder

/tmp/ipykernel_826502/44810157.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Funções

## Utilidades

In [6]:
def load_config(filename='database.ini', section='postgresql'):
    parser = ConfigParser()
    parser.read(filename)

    # get section, default to postgresql
    config = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            config[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return config
def connect(config):
    """ Connect to the PostgreSQL database server """
    try:
        # connecting to the PostgreSQL server
        with psycopg2.connect(**config) as conn:
            print('Connected to the PostgreSQL server.')
            return conn
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)

# def connect():
#     conn = None
#     try:
#         conn = postgres.connect(
#             host="localhost",
#             #database="nashville",
#             database="new-york_test",
#             user="salatiel",
#             password="root",
#             port = 5432) #8000 - LACINA PC
#                          #5432 - LOCALHOST
#     except postgres.Error as e:
#         print(e)
#     return conn

In [7]:
def closeConnection(conn):
    sucess = False
    try:
        conn.close()
        sucess = True
    except postgres.Error as e:
        print(e)
    
    return sucess

In [8]:
def executeQuery(conn, sql):
    record = None
    try:
        #print(sql)
        cur = conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor)
        cur.execute(sql)
        record = cur.fetchall()
        cur.close()
    except postgres.Error as e:
        print(e)
        cur.execute("ROLLBACK")
        cur.close()
    return record

In [9]:
def executeInsert(conn, sql):
    sucess = False
            
    try:
        #print(sql)
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
        cur.close()
        sucess = True
    except postgres.Error as e:
        print(e)
        cur.execute("ROLLBACK")
        cur.close()

    return sucess

In [10]:
#Recebe um id e retorna as categorias e checkin do lugar
def getPOIInformation(conn, business_id):
    
    sql = """
        SELECT category FROM pois_information WHERE id  = \'"""+str(business_id)+ """\'
    ;"""

    result = executeQuery(conn, sql)

    return result

In [11]:
#Encontra todos os pontos do bin centrado em um POI trazendo suas informações (categorias e checkin)
def getBinPOIsInformation(conn, business_id, bin_number):

    result = None

    sql = """
        SELECT fk_poi_id_context, name, level, checkin_count, distance_m 
        FROM bins_pois_information 
        WHERE fk_poi_id_center = \'"""+str(business_id)+"""\' AND fk_bin_number = """+str(bin_number)+""";"""

    result = executeQuery(conn, sql)

    return result


In [12]:
#Função que obtém as informações de uma view materializada referente ao OSM
def getBinOSMInformation(conn, business_id, materialized_view):
    result = None

    sql = f"""
        SELECT *
        FROM """+materialized_view+"""
        WHERE id = \'"""+str(business_id)+"""\';"""

    #print (sql)

    result = executeQuery(conn, sql)

    return result


In [13]:
#Função que obtém as estatísticas do BIN como contagem de POIs e soma dos Checkins
def getBinStats(conn, business_id, binRange):

    sql = """
    SELECT count(business_id) as quantity, sum(checkin_count) as total_checkin FROM poi WHERE business_id IN (
        SELECT fk_poi_business_id_01 FROM has_distance WHERE distance_m >= """+str(binRange[0])+""" AND distance_m < """+str(binRange[1])+"""
                                            AND fk_poi_business_id_02 = \'"""+str(business_id)+"""\'
    )
    """

    result = executeQuery(conn, sql)

    return result

In [14]:
def getCategoryInformation(data, category):
   
    occurences = 0
    checkin = 0

     #Formato de data
    #[business_id, checkin, category]
    for item in data:
        if(item[2] == category):
            occurences = occurences + 1
            checkin = checkin + item[1]
    
    return [checkin, occurences]


In [15]:
def getBinArea(conn, id, bin_number):
    sql = """
        SELECT ST_AREA(ST_Transform(bin.geom, 3857)) * 0.3048 ^ 2 area
        FROM bin
        WHERE bin.number = """+bin_number+""" LIMIT 1; ;
    """
    
    result = executeQuery(conn, sql)

    return result

## GEOC2VEC - [POI, GEO] (Todos abaixo)

In [25]:
import csv
import math
from tqdm import tqdm
def calculateBinOSMPolygon_Disco(df, mi = 10):
    
    weights = [0.7]
    #radius = (1+radius)*100
    #area = math.pi*(radius*radius)

    radius = 400
    
    print("executing radius:", radius, "m")

    #Arquivo para salvar diretamente no disco
    writers = []
    csv_files = []
    for w in weights:
        
    
        file_name = './geographic/train_files/new-york-sl-tuple-geoc2vec-pois_polygons_information-pfp-c.csv'
    
    
        csv_file = open(file_name, "w", newline='')
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow(["poi_id_center",
                         "center_poi",
                         "context_osm"])
        
        csv_files.append(csv_file)
        writers.append(writer)

    #Criando canal de comunicação com a base de dados
    try:

        config = load_config()

        connection = connect(config)

        for id_01, poi in tqdm(df.iterrows(), total=df.shape[0]):
        #for id_01, poi in df.iterrows():
            

            #print('calculating point:', id_01, 'id:', poi['business_id'])

            #[business_id, checkin, category]
            poi_information = getPOIInformation(connection, poi['POI_id'])

            
            bin_osm_information = getBinOSMInformation(connection, poi['POI_id'], 'pois_polygons_information')
            #print(bin_osm_information)
            bin_osm_building_information = getBinOSMInformation(connection, poi['POI_id'], 'pois_polygons_building_information')


            #Calculando os dois parâmetros abaixo
            #oc- total de tipos diferentes de polygons no bin
            oc = 0
            sc = 0

            #Se o bin está preenchido com alguma informação
            if (len(bin_osm_information) > 0):
                tags = list(dict(bin_osm_information[0]).keys())
                bin_osm_information = pd.DataFrame(bin_osm_information, columns = tags)

                #Verificando quantos tipos de polygons diferentes existem
                
                oc = oc + bin_osm_information.iloc[:,2:len(tags)-2][~bin_osm_information.iloc[:,2:len(tags)-2].isin(['None'])].count().sum()
                
                sc = sc + (bin_osm_information.iloc[:,2:len(tags)-2][~bin_osm_information.iloc[:,2:len(tags)-1].isin(['None'])].count(axis=1)*bin_osm_information['way_area_m']).sum()
                
                #Excluindo ids e radius
                # print("TAGS 1")
                # print(tags)

                tags = tags[2:len(tags) - 2]

            #Se o bin está preenchido com alguma informação
            if (len(bin_osm_building_information) > 0):
                tags_buildings = list(dict(bin_osm_building_information[0]).keys())
                bin_osm_building_information = pd.DataFrame(bin_osm_building_information, columns = tags_buildings)

                oc = oc + bin_osm_building_information.iloc[0]['building_count']
                
                sc = sc + bin_osm_building_information.iloc[0]['area_total']

                #Excluindo ids e radius
                # print("TAGS 2")
                # print(tags_buildings)
                tags_buildings = tags_buildings[1:]
                
            #Para evitar divisão por zero
            if(oc != 0):
                if (len(bin_osm_information) > 0):
                    for tag in tags:
                        #Percorrer cada tag
                        geographic_features = set(bin_osm_information[tag].values)

                        for feature in geographic_features:

                            if(feature != None):

                                #sf = all area o tag
                                #op = all occurences of tag 
                                
                                sf = bin_osm_information[bin_osm_information[tag] == feature]['way_area_m'].sum()
                                of = bin_osm_information[bin_osm_information[tag] == feature][tag].count()

                                SP = math.ceil((sf/sc)*mi)        
                                OP = math.ceil((of/oc)*mi)
                                
                                for idx, w in enumerate(weights):
                                    

                                    aug = int(math.ceil((w*SP) + ((1 - w)*OP)))


                                    if (aug <= 0):
                                        aug = 1

                                    #print(tag, value)
                                    name = "polygons_"+tag+"_"+feature

                                    for center_poi in poi_information: # Para cada tki
                                        #Aumentando-o pelo fator b
                                        for b in range(aug):

                                            line = [str(poi['POI_id']), 
                                                    str(center_poi['category']),
                                                    str(name)]
                                            writers[idx].writerow(line)

                #Calculando a co-ocorrência com polígonos que são unicamente prédios                    
                if (len(bin_osm_building_information) > 0):

                    for id_02, row in bin_osm_building_information.iterrows(): #Possui uma linha por bin

                            #sf = all area o tag
                            #of = all occurences of tag

                            sf = row['area_total']
                            of = row['building_count']

                            SP = math.ceil((sf/sc)*mi)        
                            OP = math.ceil((of/oc)*mi)
                            
                            for idx, w in enumerate(weights):

                                aug = int(math.ceil((w*SP) + ((1 - w)*OP)))
                                if (aug <= 0):
                                    aug = 1

                                name = 'polygons_building_yes'

                                for center_poi in poi_information: # Para cada tki
                                    #Aumentando-o pelo fator b
                                    for b in range(aug):

                                        line = [str(poi['POI_id']), 
                                                str(center_poi['category']),
                                                str(name)]
                                        writers[idx].writerow(line)
                    
        for csv_file in csv_files:
            csv_file.close()
        connection.close()

    except Exception as e:
        print(e)
        #print("Connection Failed")
    
    return None

In [26]:
import csv
import math
# Rodar pra roads e linas e realizar ajustes igual o codigo de cima
def calculateBinOSMergedRoadsLines_Disco(df, mi=20, roads=True):
    
    radius = 400
    # w = round(w, 1)
    w = 0.7
    
    if(roads):
        t_name = 'pois_roads_information'
        table = "roads"
        materialized_view = 'pois_roads_information'
    else:
        t_name = 'pois_lines_information'
        table = "lines"
        materialized_view = 'pois_lines_information'

    print("executing radius:", radius, "\tweight:", w)


    #Arquivo para salvar diretamente no disco
    file_name = './geographic/train_files/new-york-sl-tuple-geoc2vec-' + t_name + '-pfp-c.csv'
    
    csv_file = open(file_name, "w", newline='')
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(["poi_id_center",
                     "center_poi",
                     "context_osm"])

    #Criando canal de comunicação com a base de dados
    config = load_config()
    connection = connect(config)  

    if (connection != None):
        
        #centros
        for id_01, poi in df.iterrows():

            #[business_id, checkin, category]
            poi_information = getPOIInformation(connection, poi['POI_id'])


            #[business_id, checkin, category, distance_m]
            bin_osm_information = getBinOSMInformation(connection, poi['POI_id'], materialized_view)

            #Se o bin está preenchido com alguma informação
            if (len(bin_osm_information) > 0):
                tags = list(dict(bin_osm_information[0]).keys())

                bin_osm_information = pd.DataFrame(bin_osm_information, columns = tags)

                #Calculando os dois parâmetros abaixo
                #oc - total de roads/lines no bin
                oc = bin_osm_information.iloc[:,2:len(tags)-2][~bin_osm_information.iloc[:,2:len(tags)-2].isin(['None'])].count().sum()
                
                #sc - comprimento total de cada tipo de roads/lines no bin
                sc = (bin_osm_information.iloc[:,2:len(tags)-2][~bin_osm_information.iloc[:,2:len(tags)-2].isin(['None'])].count(axis=1)*bin_osm_information['length']).sum()

                
                #Excluindo ids e radius
                # print("TAGS")
                # print(tags)
                
                tags = tags[2:len(tags)-2]
            
                #As adições são feitas baseadas nos rótulos
                #Para evitar divisão por zero
                if(oc != 0):
                    for tag in tags:
                        #Percorrer cada tag
                        geographic_features = set(bin_osm_information[tag].values)

                        for feature in geographic_features:

                            if(feature != None):

                                #sp = all length o tag
                                #op = all occurences of tag
                                sf = bin_osm_information[bin_osm_information[tag] == feature]['length'].sum()
                                of = bin_osm_information[bin_osm_information[tag] == feature][tag].count()
                                
                                SP = math.ceil((sf/sc)*mi)        
                                OP = math.ceil((of/oc)*mi)

                                aug = int(math.ceil((w*SP) + ((1 - w)*OP)))

                                if (aug <= 0):
                                    aug = 1
                                
                                name = table+"_"+tag+"_"+feature

                                for center_poi in poi_information: # Para cada tki
                                    #Aumentando-o pelo fator b
                                    for b in range(aug):
                                        line = [str(poi['POI_id']), 
                                                str(center_poi['category']),
                                                str(name)]
                                        writer.writerow(line)

                                
            #break
        
        csv_file.close()
        closeConnection(connection)
        
    return None

In [27]:
import csv
import math

# TODO ajustar

def calculateBinOSMPoints_Disco(df, mi=20):
    
    radius = 400
    print("executing radius:", radius)

    #Arquivo para salvar diretamente no disco
    file_name = './geographic/train_files/new-york-sl-tuple-geoc2vec-pois_points_information-pfp-c.csv'
    
    csv_file = open(file_name, "w", newline='')
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(["poi_id_center",
                     "center_poi",
                     "context_osm"])

    #Criando canal de comunicação com a base de dados
    config = load_config()
    connection = connect(config)  

    if (connection != None):

        for id_01, poi in df.iterrows():
            #print('calculating point:', id_01, 'id:', poi['business_id'])

            #[business_id, checkin, category]
            poi_information = getPOIInformation(connection, poi['POI_id'])


            #[POI_id, checkin, category, distance_m]
            
            bin_osm_information = getBinOSMInformation(connection, poi['POI_id'], 'pois_points_information')

            
            #Se o bin está preenchido com alguma informação
            if (len(bin_osm_information) > 0):

                tags = list(dict(bin_osm_information[0]).keys())
                bin_osm_information = pd.DataFrame(bin_osm_information, columns = tags)

            
                #oc - total de tipos diferentes de points no bin
                oc = bin_osm_information.iloc[:,2:len(tags)-1][~bin_osm_information.iloc[:,2:len(tags)-1].isin(['None'])].count().sum()
                
                #Excluindo ids e radius
                tags = tags[2:len(tags)-1]
                
                #As adições são feitas baseadas nos rótulos

                #Para evitar divisão por zero
                if(oc != 0):

                    for tag in tags:
                        #Percorrer cada tag
                        geographic_features = set(bin_osm_information[tag].values)
                        
                        for feature in geographic_features:
                            
                            if(feature != None):
                                
                                of = bin_osm_information[bin_osm_information[tag] == feature][tag].count()
                                
                                OP = (of/oc)*mi
                                
                                aug = math.ceil(OP)
                                if (aug <= 0):
                                    aug = 1
                                
                                name = "points_"+tag+"_"+feature


                                for center_poi in poi_information: # Para cada tki
                                    #Aumentando-o pelo fator b
                                    for b in range(aug):

                                        line = [str(poi['POI_id']), 
                                                str(center_poi['category']),
                                                str(name)]
                                        writer.writerow(line)
        
            #break
        
        csv_file.close()
        closeConnection(connection)
        
    return None

# Geração do ITDL

In [28]:
#Quantidade de tuplas de vizinhos nos dados do yelp (center, context) considerando d = 100m
pois_file_name = '../dataset/NYC/original/all_pois.csv'
#pois_file_name = './nashville-ml-updated.csv'
#pois_file_name = './valid_pois.csv'
df = pd.read_csv(pois_file_name)
df.drop(columns=["Unnamed: 0"], axis=1, inplace=True)
print(df.shape)
df = df.dropna()
print(df.shape)
df.head()

(5099, 4)
(5099, 4)


,POI_id,POI_catname,latitude,longitude
0,49bbd6c0f964a520f4531fe3,Arts & Crafts Store,40.719810,-74.002581
1,4a43c0aef964a520c6a61fe3,Bridge,40.606800,-74.044170
2,4c5cc7b485a1e21e00d35711,Home (private),40.716162,-73.883070
3,4ce1863bc4f6a35d8bd2db6c,Home (private),40.619151,-74.035888
4,4c0ab56f7e3fc9288c1df482,Mobile Phone Shop,40.741191,-73.989663


## Iteractive Execution

In [29]:

calculateBinOSMPolygon_Disco(df, mi=5)

# Rodar duas vezes
# calculateBinOSMergedRoadsLines_Disco(df, mi=20, roads=True)
# calculateBinOSMergedRoadsLines_Disco(df, mi=20, roads=False)

# calculateBinOSMPoints_Disco(df, mi=20)
#calculateSharedContextByType_disco(n)
#calculateBinOSM_Disco(df, n, 'bins_points_information', 0.5)
#calculateBinOSM_Disco(df, n, 'bins_lines_information', 0.5)
#calculateBinOSM_Disco(df, n, 'bins_roads_information', 0.5)

executing radius: 400 m
Connected to the PostgreSQL server.


100%|██████████| 5099/5099 [01:13<00:00, 69.69it/s]


In [30]:
# Roads True
calculateBinOSMergedRoadsLines_Disco(df, mi=5, roads=True)


executing radius: 400 	weight: 0.7
Connected to the PostgreSQL server.


In [31]:
# Roads False
calculateBinOSMergedRoadsLines_Disco(df, mi=5, roads=False)

executing radius: 400 	weight: 0.7
Connected to the PostgreSQL server.


In [32]:
calculateBinOSMPoints_Disco(df, mi=5)

executing radius: 400
Connected to the PostgreSQL server.


### ITDL em Paralelo

In [ ]:
# import multiprocessing as mp
# print("Number of processors: ", mp.cpu_count())

# # Step 1: Init multiprocessing.Pool()
# pool = mp.Pool(int(mp.cpu_count()/2))

# # Step 2: `pool.apply` the `howmany_within_range()`
# bins = range(0, 4)
# #bins = [0]
# #[pool.apply(calculateBin, args=(df, n, 100, 0.3)) for n in bins]
# pool.starmap(calculateBin, [(df, n, 100, 0.7) for n in bins])

# # Step 3: Don't forget to close
# pool.close()

In [21]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

# Step 1: Init multiprocessing.Pool()
#pool = mp.Pool(int(mp.cpu_count()-4))
#pool = mp.Pool(int(mp.cpu_count()))
pool = mp.Pool(11)

# Step 2: `pool.apply` the `howmany_within_range()`
#weights = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
#weights = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
#weights = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
#n = [0, 1, 2, 3, 4, 5, 6, 7, 8]
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8]

mi = 20

pool.starmap(calculateBinOSMPolygon_dst_Disco, [(df, n,  mi) for n in bins])
#pool.starmap(calculateBinOSMRoadsLinesM_Disco, [(df, n, w) for w in weights])
pool.starmap(calculateBinOSMRoadsLines_dst_Disco, [(df, n,  mi, True) for n in bins])
pool.starmap(calculateBinOSMRoadsLines_dst_Disco, [(df, n,  mi, False) for n in bins])
pool.starmap(calculateBinOSMPoints_dst_Disco, [(df, n,  mi) for n in bins])


#LEMBRE DE EXECUTAR AO CONTRÁRIO

#pool.starmap(calculateBin_Disco, [(df, n, w) for w in weights])
#pool.starmap(calculateGUBin_Disco, [(df, n,  w) for w in weights])
#pool.starmap(calculateCGBin_Disco, [(df, n,  w) for w in weights])
#pool.starmap(calculateCGUBin_Disco, [(df, n, 'geographic', w) for w in weights])

# Step 3: Don't forget to close
pool.close()

#Para point
#calculateBinOSMPoints_Disco(df, n)

Number of processors:  20
executing radius: 100 m
executing radius: 200 m
executing radius: 300 m
executing radius: 400 m
executing radius: 500 m
executing radius: 600 m
executing radius: 700 m
executing radius: 800 m
executing radius: 900 m


100%|██████████| 22399/22399 [30:12<00:00, 12.36it/s]


executing radius: 300 m
executing radius: 400 m
executing radius: 500 m
executing radius: 100 m
executing radius: 600 m
executing radius: 700 m
executing radius: 200 m
executing radius: 800 m
executing radius: 900 m


100%|██████████| 22399/22399 [21:04<00:00, 17.71it/s]


executing radius: 100 m
executing radius: 200 m
executing radius: 300 m
executing radius: 400 m
executing radius: 500 m
executing radius: 600 m
executing radius: 700 m
executing radius: 800 m
executing radius: 900 m


100%|██████████| 22399/22399 [37:55<00:00,  9.84it/s]


executing radius: 100
executing radius: 200
executing radius: 300
executing radius: 400
executing radius: 500
executing radius: 600
executing radius: 700
executing radius: 800
executing radius: 900


 85%|████████▍ | 19001/22399 [17:55<03:08, 18.06it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 22399/22399 [24:21<00:00, 15.33it/s]


In [22]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(11)

bins = [0, 1, 2, 3, 4, 5, 6, 7, 8]
mi = 20
pool.starmap(calculateBinOSMRoadsLines_Estimated_Disco, [(df, n,  mi, True) for n in bins])
pool.starmap(calculateBinOSMRoadsLines_Estimated_Disco, [(df, n,  mi, False) for n in bins])

#LEMBRE DE EXECUTAR AO CONTRÁRIO
pool.close()


Number of processors:  20
executing radius: 100 m
executing radius: 200 m
executing radius: 300 m
executing radius: 400 m
executing radius: 500 m
executing radius: 600 m
executing radius: 700 m
executing radius: 800 m
executing radius: 900 m


100%|██████████| 22399/22399 [19:55<00:00, 18.74it/s]


executing radius: 300 m
executing radius: 400 m
executing radius: 500 m
executing radius: 600 m
executing radius: 100 m
executing radius: 200 m
executing radius: 700 m
executing radius: 800 m
executing radius: 900 m


100%|██████████| 22399/22399 [32:31<00:00, 11.48it/s]


In [3]:
print('OK')

OK


Abordagem Nayve

In [ ]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(4)
#pool = mp.Pool(int(mp.cpu_count()))

# Step 2: `pool.apply` the `howmany_within_range()`
bins = range(0, 9)

pool.starmap(calculateNayveBinOSMPolygon_Disco, [(df, n) for n in bins])
pool.starmap(calculateNayveBinOSMRoadsLines_Disco, [(df, n, True) for n in bins])
pool.starmap(calculateNayveBinOSMRoadsLines_Disco, [(df, n, False) for n in bins])
pool.starmap(calculateNayveBinOSMPoints_Disco , [(df, n) for n in bins])

# Step 3: Don't forget to close
pool.close()

Number of processors:  16
executing bin: 5
executing bin: 6
executing bin: 7
executing bin: 8
executing bin: 5
executing bin: 6
executing bin: 7
executing bin: 8


In [ ]:
calculateBinOSMPoints_Disco(df, 0)
#calculateBinOSMRoadsLines_Disco(df, 0, True)
#calculateBinOSMRoadsLines_Disco(df, 0, False)
#calculateBinOSMPolygon_Disco(df, 0)

# Teste

In [30]:
def test(a, bin_number):
    print(bin_number)
    return

In [31]:
[(i) for i in bins]

[0, 1, 2, 3, 4, 5, 6]

In [32]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(int(mp.cpu_count()))

# Step 2: `pool.apply` the `howmany_within_range()`
bins = [0, 1, 2, 3, 4, 5, 6]
a = 2

pool.starmap(test, [(a, i) for i in bins])

# Step 3: Don't forget to close
pool.close()

Number of processors:  20
6
5
0
2
1
4
3
